Desarrollo de EndPoints

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Abrimos y leemos el dataset unficado
ruta_unificado = "..\\Datasets\\unificado.parquet"
df_unificado = pd.read_parquet(ruta_unificado)

In [4]:
# Observemos las Filas, Columnas y tipo de datos
print(f'Filas : {df_unificado.shape[0]}, Variables : {df_unificado.shape[1]}')
df_unificado.info()

Filas : 8183355, Variables : 12
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8183355 entries, 0 to 8183354
Data columns (total 12 columns):
 #   Column              Dtype  
---  ------              -----  
 0   genres              object 
 1   title               object 
 2   year                object 
 3   price               float64
 4   item_id             int32  
 5   developer           object 
 6   user_id             object 
 7   recommend           bool   
 8   sentiment_analysis  int64  
 9   items_count         int64  
 10  playtime_forever    float64
 11  playtime_2weeks     float64
dtypes: bool(1), float64(3), int32(1), int64(2), object(5)
memory usage: 663.4+ MB


Se crearan conjuntos de dataset para dar respuestas a los distintos endpoints

**Endpoint 1**. Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno:

In [5]:
df_unificado["developer"].unique()

array(['Stainless Games Ltd', 'Valve', 'Outerlight Ltd.', ...,
       'Troika Games', 'Neversoft', 'Strategy First'], dtype=object)

In [26]:
ruta_archivo = "..\\Datasets\\developer.parquet"
df_developer = pd.read_parquet(ruta_archivo)
desarrollador = 'Stainless Games Ltd'
filtrado_desarrollador = df_developer[df_developer['developer'] == desarrollador]
cantidad_items = filtrado_desarrollador.groupby('year')['item_id'].count()
cantidad_gratis = filtrado_desarrollador[filtrado_desarrollador['price'] == 0.0].groupby('year')['item_id'].count()
porcentaje_gratis = (cantidad_gratis / cantidad_items * 100).fillna(0).astype(int)
diccionario = {"Cantidad de items" : cantidad_items.to_dict(),"porcentaje de contenido Free" : porcentaje_gratis.to_dict()}
diccionario
#print(cantidad_items,porcentaje_gratis )


{'Cantidad Items': {'1997': 144, '2011': 27},
 'Contenido Free': {'1997': 0, '2011': 0}}

In [7]:
df_developer = df_unificado[["year","item_id","price","developer"]]
ruta_archivo = "../Datasets/developer.parquet"               
df_developer.to_parquet(ruta_archivo, compression='snappy', index=False)
df_developer.shape

(8183355, 4)

**Endpoint 2**. Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [8]:
ruta_archivo = "..\\Datasets\\users_items.parquet"
df_userdata = pd.read_parquet(ruta_archivo)
ruta_archivo = "..\\Datasets\\steam_games.parquet"
#ruta_archivo = "..\\Datasets\\developer.parquet"
df_developer = pd.read_parquet(ruta_archivo)


In [9]:
df_userdata

,user_id,items_count,item_name,playtime_forever,playtime_2weeks,item_id
0,76561197970982479,277,Counter-Strike,6.0,0.0,10
1,76561197970982479,277,Team Fortress Classic,0.0,0.0,20
2,76561197970982479,277,Day of Defeat,7.0,0.0,30
3,76561197970982479,277,Deathmatch Classic,0.0,0.0,40
4,76561197970982479,277,Half-Life: Opposing Force,0.0,0.0,50
...,...,...,...,...,...,...
5094100,76561198329548331,7,BrainBread 2,0.0,0.0,346330
5094101,76561198329548331,7,All Is Dust,0.0,0.0,373330
5094102,76561198329548331,7,One Way To Die: Steam Edition,3.0,3.0,388490
5094103,76561198329548331,7,You Have 10 Seconds 2,4.0,4.0,521570


In [10]:
usuario = 'InstigatorAU'
df_gastos_items = df_userdata[['items_count', 'user_id', 'item_id']]
df_gastos_items

,items_count,user_id,item_id
0,277,76561197970982479,10
1,277,76561197970982479,20
2,277,76561197970982479,30
3,277,76561197970982479,40
4,277,76561197970982479,50
...,...,...,...
5094100,7,76561198329548331,346330
5094101,7,76561198329548331,373330
5094102,7,76561198329548331,388490
5094103,7,76561198329548331,521570


In [11]:
# validamos que no existan valores nulos
df_gastos_items["item_id"].isna().sum()

0

In [12]:
# Verificamos el tipo de datos em df_gastos_itmes
df_gastos_items["item_id"].dtypes

dtype('int32')

In [13]:
# Verificamso el tipo de datos en df_developer
df_developer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68023 entries, 0 to 68022
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   genres     68023 non-null  object 
 1   title      68023 non-null  object 
 2   year       68023 non-null  object 
 3   price      68023 non-null  float64
 4   developer  68023 non-null  object 
 5   item_id    68023 non-null  int32  
dtypes: float64(1), int32(1), object(4)
memory usage: 2.9+ MB


In [14]:
# Verificamso los datos que solo requerimos y asignamos a df_precio_items
df_game = df_developer
df_precio_items = df_game[["item_id", "price"]]
df_precio_items

,item_id,price
0,761140,4.99
1,761140,4.99
2,761140,4.99
3,761140,4.99
4,761140,4.99
...,...,...
68018,610660,1.99
68019,610660,1.99
68020,610660,1.99
68021,658870,4.99


In [15]:
# Vericamos si exiten duplicados y eliminamos
df_precio_items = df_precio_items.drop_duplicates(subset="item_id", keep='first')
df_precio_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27461 entries, 0 to 68021
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   item_id  27461 non-null  int32  
 1   price    27461 non-null  float64
dtypes: float64(1), int32(1)
memory usage: 536.3 KB


In [16]:
df_precio_items.reset_index(drop = True, inplace=True)

In [17]:
df_precio_items

,item_id,price
0,761140,4.99
1,643980,0.00
2,670290,0.00
3,767400,0.99
4,772540,3.99
...,...,...
27456,745400,1.99
27457,773640,1.99
27458,733530,4.99
27459,610660,1.99


In [18]:
# Realizamso un merge entre los dos dataframe df_gastos_items y df_precio_items
#df_nuevo = df_gastos_items.merge(df_precio_items, on='item_id', how='left')
df = pd.merge(df_gastos_items, df_precio_items, on="item_id",how="left")            
df

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,20,4.99
2,277,76561197970982479,30,4.99
3,277,76561197970982479,40,4.99
4,277,76561197970982479,50,4.99
...,...,...,...,...
5094100,7,76561198329548331,346330,0.00
5094101,7,76561198329548331,373330,NaN
5094102,7,76561198329548331,388490,0.00
5094103,7,76561198329548331,521570,0.00


In [19]:
# Imputemo los valores NaN con 0
df_precio_sin_nan = df['price'].fillna(0.0)
# Se borra la columna original y se concatena la columna rellena con todo el dataframe
df_gastos_items = pd.concat([df.drop('price', axis=1), df_precio_sin_nan], axis=1)
df_gastos_items

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,20,4.99
2,277,76561197970982479,30,4.99
3,277,76561197970982479,40,4.99
4,277,76561197970982479,50,4.99
...,...,...,...,...
5094100,7,76561198329548331,346330,0.00
5094101,7,76561198329548331,373330,0.00
5094102,7,76561198329548331,388490,0.00
5094103,7,76561198329548331,521570,0.00


In [20]:
# Verificamos duplicados
df_gastos_items.duplicated().sum()

23

In [21]:
# Eliminamos duplicados
df_gastos_items.drop_duplicates(keep='first',inplace=True)

In [22]:
# Verificamos duplicados
df_gastos_items.duplicated().sum()

0

In [23]:
# veamos la estructura de los datos y saquemos una copia
df_gastos_items_copia = df_gastos_items.copy()
df_gastos_items.head(10)

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,20,4.99
2,277,76561197970982479,30,4.99
3,277,76561197970982479,40,4.99
4,277,76561197970982479,50,4.99
5,277,76561197970982479,60,4.99
6,277,76561197970982479,70,9.99
7,277,76561197970982479,130,4.99
8,277,76561197970982479,300,9.99
9,277,76561197970982479,240,19.99


In [24]:
# Eminamos la variable item_id para lugo poder agrupar y sumar lo gastado por usuario
df_gastos_items = df_gastos_items.drop('item_id', axis=1)
df_gastos_items.columns

Index(['items_count', 'user_id', 'price'], dtype='object')

In [25]:
# Agrupemos por usuario y calculemos la cantidad gastada
df_gastos_items_group = df_gastos_items.groupby('user_id')['price'].sum().reset_index()
df_gastos_items_group

,user_id,price
0,--000--,397.78
1,--ace--,166.82
2,--ionex--,99.93
3,-2SV-vuLB-Kg,427.50
4,-404PageNotFound-,1509.32
...,...,...
70907,zzonci,19.98
70908,zzoptimuszz,64.98
70909,zzydrax,99.94
70910,zzyfo,813.53


In [76]:
# Calculemos la cantidad de items consumido por usuarios
df_cantidad_consumido = df_gastos_items_copia[['items_count', 'user_id']]
# Eliminamos duplicados
df_cantidad_consumido = df_cantidad_consumido.drop_duplicates(subset='user_id', keep='first')
df_cantidad_consumido

,items_count,user_id
0,277,76561197970982479
277,888,js41637
1165,137,evcentric
1302,328,Riot-Punch
1630,541,doctr
...,...,...
5093574,321,76561198320136420
5093895,4,ArkPlays7
5093899,22,76561198323066619
5093921,177,76561198326700687


In [77]:
df_gastos_items = df_cantidad_consumido.merge(df_gastos_items_group, on='user_id', how='right')
df_gastos_items

,items_count,user_id,price
0,58,--000--,397.78
1,44,--ace--,166.82
2,23,--ionex--,99.93
3,68,-2SV-vuLB-Kg,427.50
4,149,-404PageNotFound-,1509.32
...,...,...,...
70907,5,zzonci,19.98
70908,61,zzoptimuszz,64.98
70909,13,zzydrax,99.94
70910,84,zzyfo,813.53


**Endpoint 3**. Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [26]:
df_userforgenre = df_unificado[['user_id', 'playtime_forever', 'genres', 'year']]
ruta_archivo = "../Datasets/userforgenre.parquet"  
df_userforgenre.to_parquet(ruta_archivo, compression='snappy', index=False)
df_userforgenre.shape

(8183355, 4)

**Endpoint 4**. Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos) <br>
**Endpoint 5**. Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [27]:
df_bdev_devre = df_unificado[['developer', 'recommend', 'year', 'sentiment_analysis']]
ruta_archivo = '../Datasets/bdev_devre.parquet'
df_bdev_devre.to_parquet(ruta_archivo, compression='snappy', index=False)
df_bdev_devre.shape

(8183355, 4)

In [28]:
df_recomendacion = df_unificado[['item_id', 'title', 'genres', 'playtime_2weeks', 'recommend', 'sentiment_analysis', ]]
df_recomendacion.head(2)

,item_id,title,genres,playtime_2weeks,recommend,sentiment_analysis
0,282010,Carmageddon Max Pack,Action,0.0,True,1
1,282010,Carmageddon Max Pack,Action,0.0,True,1


In [29]:
# Filtrar por Recomnedado
df_recomendacion = df_recomendacion[df_recomendacion['recommend']==True]

# Filtrar por analisis de sentimiento igual a 2 (Positivo)
df_recomendacion = df_recomendacion[df_recomendacion['sentiment_analysis'] == 2]

# Borrar variable recommend
df_recomendacion = df_recomendacion.drop(['recommend'], axis=1)

In [30]:
"""
Agrupamos por juegos, titulo y genero, sumamo la cantidad de tiempo en las dos ultimas semanas
y contamos el numero de recomendaciones 
"""
df_recomendacion = df_recomendacion.groupby(['item_id', 'title', 'genres'], as_index=False).agg({
    'sentiment_analysis': 'count',
    'playtime_2weeks': 'sum',
 })

# Ordenamos de manera descendente en función del análisis de sentimiento.
df_recomendacion = df_recomendacion.sort_values(by='sentiment_analysis', ascending=False)
df_recomendacion = df_recomendacion.groupby(['item_id', 'title']).agg({
    'genres': lambda x: list(x),
    'sentiment_analysis': 'sum',
    'playtime_2weeks': 'sum'
}).reset_index()

In [31]:
ruta_archivo = '../Datasets/recomendacion.parquet'
df_recomendacion.to_parquet(ruta_archivo, compression='snappy', index=False)